# Extern block:

extern doesn't wrap the functions or variables automatically , it just calls them from the source or the header file. it is equivalent to #include <header.h>. 

Macros should be defined in extern blocks like either real functions or variables. 

semicolons must be ommited.



class declarations: because we don't want to use these classes like IO as an opaque type, we don't declare the body of the class, we just declare the name.

# extension types :

The conventional way to wrap a C++ class in Cython is with an extension type. We name it RNG to avoid clashing with the MT_RNG name, although there are ways to allow them to have the same name (see Chapter 6). Typically, a wrapper extension type has a pointer to a heap-allocated instance of the C++ class it is wrapping.


Storing a pointer to a heap-allocated C++ object in an extension type works in all instances. If the C++ class provides a nullary (no- argument) constructor, we can store a stack-allocated object directly —that is, no pointer indirection required. This removes the need to allocate and delete the instance, and there are efficiency gains as well.

An instance of a C++ class is an object that is created based on the definition of a C++ class. In C++, a class is a blueprint or template for creating objects, and an instance of a class is an actual object that is created in memory. The instance of a C++ class contains data members and member functions defined in the class. When an instance of a C++ class is created, memory is allocated for the data members of the object, and the constructor of the class is called to initialize the object's state. The instance can then be used to call member functions and access data members. When the instance is no longer needed, its destructor is called to release the memory used by the object.


## a pointer to a heap-allocated instance of the cpp class.

The __cinit__ call here uses the MT_RNG constructors based on the arguments it takes.



The "new" operator dynamically allocates memory on the heap for a new instance of an object and returns a pointer to that object.


# pointer handeling : 
Obviously, neither ptr in the first Cython-module nor val in the second are python objects. There is however a difference: a cdef variable of type double can be converted to a Python-Float automaticly by Cython via PyFloat_FromDouble - and this is exactly what happens under the hood: Cython creates a temporary Python-Float from val, passes it to print and destroys it afterwards.

There are other types which can be automatically converted to Python objects: double, float, int & similar, but also std::vector and other c++-containers.


print is a pure Python function and in order to be able to call it, the arguments must be Python objects.

Obviously, neither ptr in the first Cython-module nor val in the second are python objects. There is however a difference: a cdef variable of type double can be converted to a Python-Float automaticly by Cython via PyFloat_FromDouble - and this is exactly what happens under the hood: Cython creates a temporary Python-Float from val, passes it to print and destroys it afterwards.

There are other types which can be automatically converted to Python objects: double, float, int & similar, but also std::vector and other c++-containers.

However, there is no such automatic conversion for raw-C-pointers (char *, Py_UNICODE* and their different reincarnations being an exception - see the example later on) - because it is not possible to do in the first place, as Cython doesn't know the length of the array (is it an array at all?) to which the pointer points.

The only solution in this case: one has to manually convert the content of the memory to a python list (or to whatever python-type it corresponds), here as example the function convert_to_python:



In [39]:
%%cython 

cdef convert_to_python(double *ptr, int n):
    cdef int i
    lst=[]
    for i in range(n):
        lst.append(ptr[i])
    return lst

cdef double *ptr=[1,2,3]
print(convert_to_python(ptr,3))

UsageError: Cell magic `%%cython` not found.


In [7]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [111]:
%%writefile Binstar_h.pxd










#the libraries needed:

# distutils: language = c++
from libcpp.string cimport string
from libcpp.vector cimport vector
from libcpp cimport bool
#from libcpp cimport tWrap
from libcpp.complex cimport complex
from libc.float cimport float
from libc.stdint cimport uint_fast32_t, uint_fast64_t



#Importing the libraries: 


cdef extern from "<iostream>" namespace "std":
    cdef cppclass istringstream[T]:
        pass
    
    cdef cppclass basic_istream[T]:
        pass

    cdef cppclass basic_ostream[T]:
        pass

    ctypedef basic_istream[char] istream

    ctypedef basic_ostream[char] ostream
    
#    ctypedef istringstream[char] istringstream
    
    
    
cdef extern from "<iostream>" namespace "std::ios_base":
    cdef cppclass open_mode:
        pass
    cdef open_mode binary
    
    
# you can define other constants as needed
cdef extern from "<fstream>" namespace "std":
    cdef cppclass ofstream(ostream):
        # constructors
        ofstream(const char*) except +
        ofstream(const char*, open_mode) except+
        
cdef extern from "<fstream>" namespace "std":
    cdef cppclass ifstream(ostream):
        # constructors
        ifstream(const char*) except +
        ifstream(const char*, open_mode) except+
        


cdef extern from "<random>" namespace "std" nogil:
    cdef cppclass random_device:
        ctypedef uint_fast32_t result_type
        random_device() except +
        result_type operator()() except +

        




#calling the cpp source files:
#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/src/binstar/binstar.cpp":
#     pass

#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/src/general/IO.cpp":
#     pass

#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/src/general/utils/utilities.cpp":
#     pass

#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/src/star/star.cpp":
#     pass

#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/src/binstar/Processes.cpp":
#    pass

#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/src/binstar/BinaryProperty.cpp":
#    pass

#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/src/star/property.cpp":
#    pass
#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/src/general/utils/sevnlog.cpp":
#    pass






#calling classes or other dependencies that we need:

cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/include/general/IO.h" : 
     cdef cppclass IO:
        IO() except +
        IO(int , char) except +


#cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/include/general/utils/utilities.h" namespace "utilities":
#    pass
        
        
        
    

cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/include/star/star.h":
     cdef cppclass Star:
        Star() except +
        Star(IO , vector[string] , size_t , bool , unsigned long) except +







cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/include/binstar/Processes.h":
    cdef cppclass Process:
        Process() except +
        
        

cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/include/binstar/BinaryProperty.h":
    cdef cppclass BinaryProperty:
        BinaryProperty() except +

cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/include/star/property.h":
    cdef cppclass Property:
        Property() except +

    
cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/include/general/utils/sevnlog.h" namespace "sevnstd":
    cdef cppclass SevnLogging:
        SevnLogging() except +
        



cdef extern from "/Users/melikakeshavarz/Desktop/sevndevel/include/binstar/binstar.h":
    cdef cppclass Binstar:
        Binstar() except +
        Binstar(IO , vector[string] , size_t , unsigned long) except +
        





        #public:
            
    #MEMBERS:
        ctypedef bool bse_evolution
        ctypedef unsigned int sn_explosion
        bool broken 
        bool onesurvived  # true if just one star survived else either both stars are present or no stars are present
        bool empty
        bool break_at_remnant  # if true stop the evolution when both stars are remnant
        bool break_at_broken # if true stop the evolution when the binary breaks
        bool repeatstep  #!Control if it is needed to repeat a step of the binary evolution. It is mostly used in BTimestep::evolve
        bool mix # If true the binary is mixing
        bool comenv # If true the binary is doing a  ce
        bool is_swallowed[2] # Array of bool corresponding to the ID of stars in binary. If true, the star has been swallowed
        double last_Timestep # it stores the last value of the Timestep used in the evolution 
        bool print_all_steps  #  If true print in output all the steps.
        bool print_per_phase  # If true print in output in each phase change
        bool print_only_end # If true do not print intermediate steps 
        bool print_events # If true print all the events 
        bool print_rlo # If true print all the steps during a rlo 
        bool disable_e_check # If true, the e_check in adpative timestep is disabled 
        bool disable_a_check # If true, the a_check in adpative timestep is disabled 
        bool disable_DM_check # If true, the DM_check (from binary evolution) in adpative timestep is disabled 
        bool disable_OmegaRem_NS_check # If true, the OmegaRem_NS_check (from binary evolution) in adpative timestep is disabled 
        bool disable_stellar_rotation_check # If true, the OmegaSpin (from binary evolution) in adpative timestep is disabled 
        bool force_tiny_dt # If true, it forces the next Binary time step to be equal to utilities::TINY 

        #Other variables
        double CE_E_tomatch  # Envelope binding energy of the system after coalesce during a CE  

        #Guard
        bool evolution_step_completed



        #Methods:
        inline double getp(const size_t &id) 
        inline double getp_0(const size_t &id) 
        inline string get_name() #this was std::string get_name()
        inline size_t get_ID() 
        inline Star *getstar(const size_t &id)  
        inline Process *getprocess(const size_t &id) 
        inline const vector[Process*]& getprocesses() #this was std::vector<Process*>&
        inline const vector[double] & getstate() #this was std::vector<double> &



        inline double get_last_Timestep()
        inline unsigned  int get_id_more_evolved_star()
        inline Star *get_more_evolved_star()


        double Radx(size_t starID)



        inline double get_svpar_num(string name)
        inline string get_svpar_str(string name)
        inline bool get_svpar_bool(string name)
        inline unsigned long get_rseed()




        void evolve() #this is a virtual function!




        inline bool is_process_ongoing(const size_t &id)
        inline bool processes_alarm(const size_t &id) 

        bool process_alarm_any()#const


        inline void reset_all_processes_alarms()


        inline void reset_events()



        inline double get_event_from_processes()


        bool isoutputtime()

        inline bool printall()
        inline bool notprint()


        void print()



        void print_failed(bool include_in_output=false)


        inline void print_to_log(string& message)


        void init(const vector[string] &params)


        inline vector[double]  get_zams()

        inline vector[double]  get_Z()

        inline void set_tf(const double a, const char* file, const int line)

        inline void set_dtout(const double a, const char* file, const int line)

        inline void evolution_step_done()

        inline void evolution_guard(const char *file_input = nullptr, int line_input = -1)

        inline double  get_tf()

        inline double get_dtout()

        inline string get_id_name()


        sn_explosion check_accretion_on_compact(size_t donorID, size_t accretorID, double DMaccreted)

        void check_accretion_on_compact()



        bool check_and_set_broken()



        void check_and_set_bavera_xspin()
        void check_if_applied_bavera_xspin()
        double SetXspinBavera(const double, const double)








        #Members:
        double tf
        double dtout
        vector[double] state # this was std::vector<double>
        vector[double] combinedstate
        vector[vector[double]] allstates 


        #Methods
        void synchronise_dt_star()
        void check_and_sync_sse()
        void check_and_sync_bse()
        void check_nakedHe_or_nakedCO_after_binary_evolution()
        void check_AngMomSpin_after_binary_evolution()
        void limit_and_correct_mass_transfer_for_donor_from_binary(Star *donor, Star *accretor,
                                                               size_t mass_limiting_property_id)
        bool check_and_set_QHE(Star *accretor)
        void update_derived_properties_star(Star *s)
        inline void reset_evolution_flags()





        SevnLogging svlog




       #MEMBERS
        vector[Process*] process  # List with all the pointers to the processes initialised in static_main.h
        vector[BinaryProperty*] property  # List with all the pointers to the binary properties initialised in static_main.h
        Star *star[2] #Star *star[2]={nullptr, nullptr}  # Pointers to the stars in the binary system, initiliased to nullptr
        IO *io     #Pointer to the IO object
        size_t ID  #ID of the binary (it depends on the position of this system in the list of binary
        vector[string] init_params; #vector of string loaded from the list of binary
        string name   #Random name of the binary*/
        unsigned long rseed  #Random seed*/
    

        #METHODS
        bse_evolution evolve_binary()

        inline void resynch(const double &dt)

    

        #Method to clean the heap from all the heap allocated members of this class

        void default_destructor()


        # Call the constructor of the two stars with parameters params_star1 and params_star2.
        # The parameters are Mass, Z, Spin, sn_type, t initial, t final, dt print output.
        # param params_star1  Mass, Z, Spin, sn_type, t initial, t final, dt print output of the first star.
        # param params_star2 Mass, Z, Spin, sn_type, t initial, t final, dt print output of the second star
        #note The random seeds of the stars are set to the same random seed of the binary.
     
        void call_stars_constructor(vector[string] &params_star1, vector[string] &params_star2)

        # Transform the parameter in input it the stardard form needed from the Star constructor and then call the stars constructor
        # NB: This is the version that I like more, but it is different with respect the
        # old SEVN V1 formalism (implemented in init_star_legacy)
        # The constructor is called in the standard way from a init param vector with:
        # Mass, Z, Spin, sn_type, t initial, t final, dt print output.
        # @param params A vector of string with the following orderd parameters:
        # Mass1, Z1, Spin1, sn_type1,  t_1 initial, Mass2, Z2, Spin2, sn_type2,  t_2 initial,  bin_separation, bin_eccentricity, t_final, dt_out
     
        void init_stars(const vector[string] &params)


        # Transform the parameter in input in the standard format needed from the Star constructor and then call the stars constructor.
        # The input format is in the old SEVN1 style.
        # The constructor is called in the standard way from a init param vector with:
        # Mass, Z, Spin, sn_type, t initial, t final, dt print output.
        # @param params A vector of string with the following orderd parameters:
        # Mass1, Mass2, Z1, Z2, Spin1, Spin2, bin_separation, bin_eccentricity, t_final, t_ini, tstep,  sn_type1, sn_type2, dt_out
        # NB: tstep not used at the moment

        void init_stars_legacy(const vector[string] &params)


        # Initialise the Semimajor checking the value
        # @param a_ini Initial value of the Semimajor axis, it should be larger than 0 or the functions throw a critical error
       
        inline void set_initial_semimajor(double a_ini)

    
        # Initialise the Semimajor checking the value
        # param a_ini string with the initial semimajor axis, if the string cannot be transformed to a number the function throw an io error.
        # a_ini has to be larger than 0 or the functions throw a critical error
        #
        inline void set_initial_semimajor(string a_ini)


        # Initialise the Eccentricity checking the value
        # param ecc_ini  Initial value of the eccentricity, it should be lower than 1 and no lower than 0
     
        inline void set_initial_eccentricity(double ecc_ini)


        # Initialise the Eccentricity checking the value
        # param ecc_ini string with the initial eccentricity, if the string cannot be transformed to a number the function throw an io error.
        # The initial value of the eccentricity has to  be lower than 1 and no lower than 0

        inline void set_initial_eccentricity(string ecc_ini)


        # Initialise the binary parameters.
        # param params  A vector of string with the following ordered parameters:
        #          if binput_mode is legacy: Mass1, Mass2, Z1, Z2, Spin1, Spin2, bin_separation, bin_eccentricity, t_final, t_ini, tstep  sn_type1, sn_type2, dt_out
        #          if binput_mode is new: Mass1, Z1, Spin1, sn_type1,  t_1 initial, Mass2, Z2, Spin2, sn_type2,  t_2 initial,  bin_separation, bin_eccentricity, t_final, dt_out

        inline void init_binary_properties(const vector[string] &params)



        # In this function we set the parameters that are already set in each star when we call init_stars or init_stars_legacy.
        # These parameters are: break_at_broken, break_at_remnant, print_all_steps and print_per_phase, tf, dtout.
        # The function just check that  the two stars store the same value for the parameters to set. It this is not true it throws a critical error.
        # note this function should be called after the initialisation of the stars (init_stars or init_stars_legacy).
   
        void  init_other_params()


        # Set time when the tf in input is equal to broken
        # param tf

        void set_break_at_broken(string &tf)

        inline void set_rseed(const unsigned long a, const char* file, const int line)


        # Check if params in input has the expected number of parameters as defined in INPUTMAPBIN and INPUTMAPBIN_PARAM (lookup_and_phases.cpp)
        # param params params to pass to the init function.

        inline void check_init_param(const vector[string] &params)



















Overwriting Binstar_h.pxd


In [130]:
%%writefile Binstar.pyx
# cython: no_override=True

# distutils: language = c++


from Binstar_h cimport *

cdef class pIO:
    cdef IO* C_IO
    
    
    def __init__(self):
        self.C_IO = new IO()
        
        
    def __dealloc__(self):
        del self.C_IO
        
        

cdef class pSevnLogging:
    
    cdef SevnLogging* C_SevnLogging
    
    
    def __init__(self):
        self.C_SevnLogging = new SevnLogging()
        
        
    def __dealloc__(self):
        del self.C_SevnLogging 
        
        

    
cdef class pProcess:

    cdef Process* C_Process
    
    
    def __init__(self):
        self.C_Process = new Process()
        
    def dealloc(self):
        del self.C_Process
        
        
        
cdef class pBinaryProperty:

    cdef BinaryProperty* C_BinaryProperty
    
    
    def __init__(self):
        self.C_BinaryProperty = new BinaryProperty()
        
    def dealloc(self):
        del self.C_BinaryProperty
        
        
        
cdef class pStar:

    cdef Star* C_Star
    
    
    def __init__(self):
        self.C_Star = new Star()
        
    def dealloc(self):
        del self.C_Star
        
        
     
        
        









cdef class pBinstar:

    cdef Binstar* C_inst   # Hold a heap-allocated C++ instance which we're wrapping



    def __cinit__(self):
        self.C_inst = new Binstar()
        if self.C_inst == NULL:
            raise MemoryError('Not enough memory!')


    def __dealloc__(self):
        if self.C_inst != NULL:
            del self.C_inst


Overwriting Binstar.pyx


In [136]:
%%writefile setup.py
from distutils.core import setup, Extension
from Cython.Build import cythonize
import glob

path = "/Users/melikakeshavarz/Desktop/sevndevel"

#src_files=glob.glob(path +"/src/binstar/binstar.cpp",recursive=True)


ext= Extension("Binstar",sources=["Binstar.pyx"],
               language="c++",
               extra_compile_args=["-std=c++11"],
               extra_link_args=['-std=c++11'],
               include_dirs = [path +'/include',path+'/include/star',path +'/include/star/lambdas',
                               path +'/include/star/procs/kicks',path +'/include/star/procs/neutrinomassloss',path +'/include/star/procs/pairinstability',
                               path + '/include/star/procs/supernova',
                               path +'/include/general/utils',path +'/include/general/',path +'/include/binstar/procs',
                               path +'/include/star/lambdas',path +'/include/star/procs',path +'/include/binstar'])



setup(name="Binstar",ext_modules=cythonize(ext,compiler_directives={'language_level' : "3"}))

Overwriting setup.py


In [128]:
import os

os.environ["CC"] = "gcc-12"
os.environ["CXX"] = "g++-12"


In [138]:
!python3 setup.py build_ext --inplace 

Binstar.cpp: In function 'void __pyx_pf_3Bin_8pBinstar_2__dealloc__(__pyx_obj_3Bin_pBinstar*)':
Binstar.cpp:2806:5: warning: deleting object of polymorphic class type 'Binstar' which has non-virtual destructor might cause undefined behavior []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wdelete-non-virtual-dtor-Wdelete-non-virtual-dtor]8;;]
 2806 |     delete __pyx_v_self->C_inst;
      |     ^~~~~~~~~~~~~~~~~~~~~~~~~~~
ld: warning: dylib (/opt/homebrew/Cellar/gcc/12.2.0/lib/gcc/current/libstdc++.dylib) was built for newer macOS version (13.0) than being linked (11.1)


In [139]:
from Binstar import Binstar

ImportError: dlopen(/Users/melikakeshavarz/Desktop/Binstar.cpython-310-darwin.so, 0x0002): symbol not found in flat namespace '__ZN10BWorldtime2IDE'